# ARC Test on Llama 2

In [1]:
import os
import json
import replicate
import re
import pandas as pd
from tqdm import tqdm
from transformers import LlamaTokenizer

## Connection to the API

In [2]:
os.environ['REPLICATE_API_TOKEN'] = 'r8_TPbThLzct109l1IBPqPDWoLB1GdVPPD4f3CaT'

## Number of Tokens

Since the number of tokens which can be fed to Llama 2 is limited to 4096, we need to count the number of tokens present in each prompt we are generating before sending them. System prompt is always sent to the model together with the main prompt. Let's calculate the number of tokens in system prompt:

In [3]:
system_prompt = """
You will be given a list of input-output pairs labeled "Case 0" "Case 1" and so on.

Each input and output is a grid of numbers representing a visual grid. There is a SINGLE rule that transforms each input
grid to the corresponding output grid.

The pattern may involve counting or sorting objects (e.g. sorting by size) comparing numbers (e.g. which shape or symbol
appears the most? Which is the largest object? Which objects are the same size?) or repeating a pattern for a fixed number
of time.

There are other concepts that may be relevant.

- Lines rectangular shapes

- Symmetries rotations translations.

- Shape upscaling or downscaling elastic distortions.

- Containing / being contained / being inside or outside of a perimeter.
- Drawing lines connecting points orthogonal projections.

- Copying repeating objects.

You should treat cells with 0 as empty cells (backgrounds) .

Please generate the Output grid that corresponds to the last given Input grid using the transformation rule you induced
from the previous input-output pairs. Just give the output with no further words.
"""

We will use the below tokenizer througout the code:

In [4]:
# Initialize the tokenizer
tokenizer = LlamaTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
# Encode the system prompt
encoded_prompt = tokenizer.encode(system_prompt)

# Calculate the number of tokens
num_tokens = len(encoded_prompt)

print(f"The number of tokens in the system prompt is {num_tokens}.")

The number of tokens in the system prompt is 269.


Therefore, for the main prompt we can tolerate 4096 - 269 = 3827 ~ 3820 tokens.

A couple of useful functions for the rest of the code:

In [6]:
def keep_numbers_only(input_string):         # this is a function  to create a serial version of the results
    # Use regular expression to remove non-numeric characters
    numbers_only = re.sub(r'[^0-9]', '', input_string)
    return numbers_only

In [7]:
def new_content(result , file_content ):

  new_content = re.sub(r"[^0-9\s\[\]\n]", "", result).replace("\n", "")

  file_content += new_content + '\n [INST] The last output you produced is wrong, try again by predicting another output for the last input given. [/INST]' + '\n output:'

  return file_content

# Run The Full Test on The Model

In [20]:
Comparison = []

folder_path = 'New_Prompts/'

# Get list of files with their sizes
files_with_sizes = [(f, os.path.getsize(os.path.join(folder_path, f))) for f in os.listdir(folder_path)]

# Filter files to keep only those smaller than 5 KB (5 * 1024 bytes)
filtered_files = [(name, size) for name, size in files_with_sizes if size < 5 * 1024]

# Sort filtered files based on their sizes in descending order
sorted_files = sorted(filtered_files, key=lambda x: x[1], reverse=True)

# Extract only the filenames from the sorted list
sorted_filenames = [f[0] for f in sorted_files]

print(len(sorted_filenames))

480


In [25]:
processed_files = set()

# Collect the names of already processed files
for entry in Comparison:
    processed_files.add(entry[0])

In [26]:
processed_files

{'AboveBelow10_0.json',
 'AboveBelow10_1.json',
 'AboveBelow10_2.json',
 'AboveBelow1_0.json',
 'AboveBelow1_1.json',
 'AboveBelow1_2.json',
 'AboveBelow2_0.json',
 'AboveBelow2_1.json',
 'AboveBelow2_2.json',
 'AboveBelow7_0.json',
 'AboveBelow7_1.json',
 'AboveBelow7_2.json',
 'AboveBelow8_0.json',
 'AboveBelow8_1.json',
 'AboveBelow8_2.json',
 'AboveBelow9_0.json',
 'AboveBelow9_1.json',
 'AboveBelow9_2.json',
 'Center1_2.json',
 'Center3_0.json',
 'Center3_1.json',
 'Center3_2.json',
 'Center5_0.json',
 'Center5_1.json',
 'Center5_2.json',
 'Center6_0.json',
 'Center6_1.json',
 'Center6_2.json',
 'Center8_0.json',
 'Center8_1.json',
 'Center8_2.json',
 'Center9_1.json',
 'CleanUp1_0.json',
 'CleanUp3_0.json',
 'CleanUp3_1.json',
 'CleanUp3_2.json',
 'CleanUp4_0.json',
 'CleanUp4_1.json',
 'CleanUp4_2.json',
 'CleanUp5_0.json',
 'CleanUp5_1.json',
 'CleanUp5_2.json',
 'CleanUp6_0.json',
 'CleanUp6_1.json',
 'CleanUp6_2.json',
 'CleanUp7_0.json',
 'CleanUp7_1.json',
 'CleanUp7_2.json

In [27]:
for f in tqdm(sorted_filenames, desc="Processing files", unit="file"):
    
    # Check if the file has already been processed
    if f in processed_files:
        print(f"{f} already processed. Skipping...")
        continue
    
    ignore_file = False
    
    prompt_path = os.path.join('New_Prompts/', f)
    correctanswer_path = os.path.join('New_Correct_Answers/', f)

    with open(prompt_path, 'r') as file:
        prompt_content = file.read()

    with open(correctanswer_path, 'r') as file:
        answer_content = file.read()

    prompt = ' """ ' + prompt_content + ' """ '
    
    trial = 0

    for i in range(3):
        encoded_prompt = tokenizer.encode(prompt)
        
        if len(encoded_prompt) < 3810:
            trial += 1

            output_generator = replicate.run(
                "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d",
                input={
                    "prompt": prompt,
                    "system_prompt": system_prompt,
                    "max_new_tokens": 2048,
                    "temperature": 0.5
                }
            )

            output_list = list(output_generator)
            output_string = " ".join(output_list)
            result = output_string

            model_answer = keep_numbers_only(result)
            correct_answer = keep_numbers_only(answer_content)

            if model_answer == correct_answer:
                break

            prompt_content = new_content(result, prompt_content)
            prompt = ' """ ' + prompt_content + ' """ '
        else:
            if trial == 0:
                ignore_file = True
                break
            else:
                break
    
    if ignore_file:
        Comparison.append([f, "NA", "NA", "NA", "NA", trial])
    else:
        Comparison.append([f, model_answer, correct_answer, model_answer == correct_answer,
                           len(model_answer) == len(correct_answer), trial])

    processed_files.add(f)
    print(Comparison[-1])


Processing files:   0%|          | 0/480 [00:00<?, ?file/s]

ExtendToBoundary2_1.json already processed. Skipping...
AboveBelow2_2.json already processed. Skipping...
ExtendToBoundary2_2.json already processed. Skipping...
InsideOutside4_0.json already processed. Skipping...
InsideOutside4_1.json already processed. Skipping...
InsideOutside4_2.json already processed. Skipping...
ExtendToBoundary2_0.json already processed. Skipping...
AboveBelow2_1.json already processed. Skipping...
AboveBelow2_0.json already processed. Skipping...
InsideOutside5_2.json already processed. Skipping...
InsideOutside5_1.json already processed. Skipping...
HorizontalVertical8_1.json already processed. Skipping...
InsideOutside5_0.json already processed. Skipping...
ExtendToBoundary9_0.json already processed. Skipping...
HorizontalVertical8_0.json already processed. Skipping...
ExtendToBoundary9_1.json already processed. Skipping...
HorizontalVertical4_2.json already processed. Skipping...
Count2_2.json already processed. Skipping...
HorizontalVertical8_2.json alread

Processing files:  52%|█████▏    | 249/480 [00:07<00:06, 35.51file/s]

['ExtractObjects4_2.json', '333', '33333333333333333333', False, False, 3]


Processing files:  52%|█████▏    | 250/480 [00:38<00:48,  4.77file/s]

...

['Center2_1.json', '6', '6', True, True, 1]


Processing files: 100%|█████████▉| 479/480 [56:29<00:05,  5.99s/file]

['Center2_2.json', '0', '0', True, True, 2]


Processing files: 100%|██████████| 480/480 [56:32<00:00,  7.07s/file]

['Center2_0.json', '5', '5', True, True, 2]


In [28]:
Comparison

[['ExtendToBoundary2_1.json',
  '55555555555555555555555555555',
  '555555555555555555550000003000000000055055555355555550005505000030000005000550500553555500500055050050300050050005505005555555005000550500000000000500055055555555555550005500000000000000000550000000000000000055555555555555555555',
  False,
  False,
  1],
 ['AboveBelow2_2.json',
  '000000000000000055550000000000555500000000000000',
  '00000000000000000000000000000000000000000000000000000000000000000000000000000000033000000000000330000000555505500000005555055000000000000000000000000000000000000000000000000000000000',
  False,
  False,
  1],
 ['ExtendToBoundary2_2.json',
  '0000000000000000999999999999009000000000090090999990000900900',
  '000000000000000000999999999990000999990999990000999990999990000999010009990000999010009990000999010009990000999010009990000999010009990000999999999990000999999999990000999999999990000000000000000',
  False,
  False,
  1],
 ['InsideOutside4_0.json',
  '00000000000000000000000000000088888

In [29]:
df = pd.DataFrame(Comparison, columns=['Task ', 'Raw Model Answer', 'Correct Answer' , 'Model Answer Evaluation' , 'Model Answer Dimension Evaluation', 'Number of Trials' ])

In [30]:
df

,Task,Raw Model Answer,Correct Answer,Model Answer Evaluation,Model Answer Dimension Evaluation,Number of Trials
0,ExtendToBoundary2_1.json,55555555555555555555555555555,5555555555555555555500000030000000000550555553...,False,False,1
1,AboveBelow2_2.json,000000000000000055550000000000555500000000000000,0000000000000000000000000000000000000000000000...,False,False,1
2,ExtendToBoundary2_2.json,0000000000000000999999999999009000000000090090...,0000000000000000009999999999900009999909999900...,False,False,1
3,InsideOutside4_0.json,0000000000000000000000000000008888888888000080...,0000000000000000000000000000008888888888000080...,False,False,1
4,InsideOutside4_1.json,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,False,False,1
...,...,...,...,...,...,...
475,Copy1_0.json,8080000008000800000000808080088080000008000800...,808808000000080080000000808808,False,False,3
476,Copy1_1.json,666666666666666666666666666666666666666666,66666666666666666666,False,False,3
477,Center2_1.json,6,6,True,True,1
478,Center2_2.json,0,0,True,True,2


In [31]:
excel_filename = 'results_full.xlsx'
df.to_excel(excel_filename, index=False)